In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lmtracker/__results__.html
/kaggle/input/lmtracker/__notebook__.ipynb
/kaggle/input/lmtracker/__output__.json
/kaggle/input/lmtracker/custom.css
/kaggle/input/lmtracker/graph_data_20251119174904/malicious_test_path.pt
/kaggle/input/lmtracker/graph_data_20251119174904/graph_data_torch.pt
/kaggle/input/lmtracker/graph_data_20251119174904/malicious_train_val_path.pt
/kaggle/input/lmtracker/graph_data_20251119174904/computer2nodeid.pt
/kaggle/input/lmtracker/graph_data_20251119174904/process2nodeid.pt
/kaggle/input/lmtracker/graph_data_20251119174904/user2nodeid.pt
/kaggle/input/lmtracker/graph_data_20251119174904/normal_path.pt
/kaggle/input/lmtracker/graph_data_20251119174904/graph_data_dgl.bin
/kaggle/input/lmtracker/graph_data_20251119174904/malicious_CUC_path.pt


In [2]:
# 1. Gỡ hết các thứ cũ có thể gây xung đột
!pip uninstall -y torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric pyg-lib -q

# 2. Cài đúng phiên bản torch + cu121 hiện tại của Colab (rất quan trọng)
!pip install torch==2.4.0+cu121 torchvision==0.19.0+cu121 torchaudio==2.4.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121 -q

# 3. Cài PyG + các package phụ thuộc từ wheel chính thức (dùng torch 2.4.0 mới nhất)
!pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cu121.html -q
!pip install torch_geometric -q

# 4. Cài các package phụ thường cần thêm
!pip install pyg_lib torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cu121.html -q

# 5. (Tuỳ chọn) Nếu vẫn còn lỗi cuGraph → downgrade tạm để khỏi conflict
!pip install libcugraph-cu12==25.2.* pylibcugraph-cu12==25.2.* libraft-cu12==25.2.* rmm-cu12==25.2.* --extra-index-url=https://pypi.nvidia.com -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.1/799.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 100.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch_scatter, torch_sparse, torch_cluster, torch_spline_conv
import torch_geometric
from torch_sparse import SparseTensor

print("✅ torch_scatter OK")
print("✅ torch_sparse OK")
print("✅ torch_cluster OK")
print("✅ torch_spline_conv OK")
print("PyG:", torch_geometric.__version__)

✅ torch_scatter OK
✅ torch_sparse OK
✅ torch_cluster OK
✅ torch_spline_conv OK
PyG: 2.7.0


In [4]:
import random
import os
import numpy as np
import torch

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Gọi hàm này trước khi chạy bất cứ thứ gì khác
seed_everything(42)

In [5]:
import os

REQUIRED_FILES = [
    "graph_data_torch.pt",
    "normal_path.pt",
    "malicious_CUC_path.pt",
    "malicious_train_val_path.pt",
    "malicious_test_path.pt",
    "computer2nodeid.pt",
    "user2nodeid.pt",
]

def find_lmtracker_dir(base_dir="/kaggle/input/lmtracker"):
    """
    Tìm thư mục chứa graph_data_torch.pt và toàn bộ file output lmTracker.
    Có thể nằm trực tiếp bên trong hoặc trong thư mục con.
    """
    # TH1: file nằm trực tiếp
    if "graph_data_torch.pt" in os.listdir(base_dir):
        return base_dir

    # TH2: file nằm trong thư mục con
    for root, dirs, files in os.walk(base_dir):
        if "graph_data_torch.pt" in files:
            return root

    return None


print("🔍 Searching for lmTracker output folder...")

lm_dir = find_lmtracker_dir()

if lm_dir is None:
    raise FileNotFoundError(
        "❌ Không tìm thấy thư mục chứa graph_data_torch.pt trong /kaggle/input/lmtracker.\n"
        "➡ Hãy kiểm tra lại bạn đã attach đúng dataset lmTracker hay chưa."
    )

print(f"✅ Found lmTracker directory:\n{lm_dir}\n")

# In toàn bộ file có trong thư mục đó
print("📂 Files inside this directory:")
for f in sorted(os.listdir(lm_dir)):
    print("   ├─", f)
print("\n")

# Check thiếu file nào không
print("📌 Checking required files...\n")
missing_files = []

for fname in REQUIRED_FILES:
    fpath = os.path.join(lm_dir, fname)
    if not os.path.isfile(fpath):
        missing_files.append(fname)
        print(f"❌ Missing: {fname}")
    else:
        print(f"✅ Found: {fname}")

if missing_files:
    print("\n⚠️ WARNING: Dataset thiếu file sau:")
    for f in missing_files:
        print("   -", f)
    print("\n❗ Code MetaPath2Vec sẽ lỗi nếu thiếu các file này.")
else:
    print("\n🎉 All required files exist! Dataset is READY.")

🔍 Searching for lmTracker output folder...
✅ Found lmTracker directory:
/kaggle/input/lmtracker/graph_data_20251119174904

📂 Files inside this directory:
   ├─ computer2nodeid.pt
   ├─ graph_data_dgl.bin
   ├─ graph_data_torch.pt
   ├─ malicious_CUC_path.pt
   ├─ malicious_test_path.pt
   ├─ malicious_train_val_path.pt
   ├─ normal_path.pt
   ├─ process2nodeid.pt
   ├─ user2nodeid.pt


📌 Checking required files...

✅ Found: graph_data_torch.pt
✅ Found: normal_path.pt
✅ Found: malicious_CUC_path.pt
✅ Found: malicious_train_val_path.pt
✅ Found: malicious_test_path.pt
✅ Found: computer2nodeid.pt
✅ Found: user2nodeid.pt

🎉 All required files exist! Dataset is READY.


# GraphSAGE Model Implementation for Heterogeneous Graphs

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from typing import Dict
from torch_geometric.nn import SAGEConv, HeteroConv

class HeteroGraphSAGE(nn.Module):
    """
    GraphSAGE cho đồ thị dị thể (Computer, User, Process, ...).

    - Mỗi node type có 1 embedding riêng.
    - Mỗi edge type dùng 1 SAGEConv (GraphSAGE).
    - Dùng HeteroConv để gộp message từ nhiều edge type.
    - Trả về x_dict: embedding out_channels cho từng node type.
    """

    def __init__(
        self,
        metadata,
        num_nodes_dict: Dict[str, int],
        hidden_channels: int = 128,
        out_channels: int = 128,
        num_layers: int = 2,
        dropout: float = 0.1,
    ):
        super().__init__()
        self.metadata = metadata
        self.node_types, self.edge_types = metadata
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.num_layers = num_layers
        self.dropout = dropout

        # 1) Trainable embedding cho từng loại node
        self.embeddings = nn.ModuleDict()
        for nt in self.node_types:
            n = num_nodes_dict[nt]
            self.embeddings[nt] = nn.Embedding(n, hidden_channels)

        # 2) HeteroConv gồm nhiều SAGEConv cho từng edge type
        self.convs = nn.ModuleList()
        for layer in range(num_layers):
            conv = HeteroConv(
                {
                    # KHÔNG dùng (-1, -1) để tránh LazyModule
                    edge_type: SAGEConv((hidden_channels, hidden_channels), hidden_channels)
                    for edge_type in self.edge_types
                },
                aggr="sum",  # sum/mean/max: aggregate giữa các edge type
            )
            self.convs.append(conv)

        # 3) Linear chiếu ra out_channels cho từng node type
        self.lin_out = nn.ModuleDict()
        for nt in self.node_types:
            self.lin_out[nt] = nn.Linear(hidden_channels, out_channels)

        self.reset_parameters()

    def reset_parameters(self):
        for nt in self.node_types:
            nn.init.xavier_uniform_(self.embeddings[nt].weight)
        for conv in self.convs:
            for m in conv.modules():
                if isinstance(m, SAGEConv):
                    m.reset_parameters()
        for nt in self.node_types:
            nn.init.xavier_uniform_(self.lin_out[nt].weight)
            if self.lin_out[nt].bias is not None:
                nn.init.zeros_(self.lin_out[nt].bias)

    def forward(self, edge_index_dict):
        """
        edge_index_dict: data.edge_index_dict (đã .to(device))
        Trả về: dict[node_type] -> tensor [num_nodes_type, out_channels]
        """
        # 1) Init embedding cho từng node type: x_dict[nt] ∈ [N_nt, hidden]
        x_dict = {}
        for nt in self.node_types:
            num_nodes = self.embeddings[nt].num_embeddings
            idx = torch.arange(num_nodes, device=self.embeddings[nt].weight.device)
            x_dict[nt] = self.embeddings[nt](idx)

        # 2) Message passing qua từng layer
        for i, conv in enumerate(self.convs):
            x_dict = conv(x_dict, edge_index_dict)  # dict[nt] ∈ [N_nt, hidden]
            if i < len(self.convs) - 1:
                for nt in x_dict:
                    x_dict[nt] = F.relu(x_dict[nt])
                    x_dict[nt] = F.dropout(x_dict[nt], p=self.dropout, training=self.training)

        # 3) Linear + L2 normalize
        out_dict = {}
        for nt in x_dict:
            h = self.lin_out[nt](x_dict[nt])          # [N_nt, out_channels]
            h = F.normalize(h, p=2, dim=-1)           # chuẩn hoá L2
            out_dict[nt] = h
        return out_dict


# GraphSAGE Training Script

In [7]:
import os, datetime, time, math, random
import torch
import torch.nn.functional as F
from tqdm import tqdm
from collections import defaultdict

## 1. Load graph lmtracker & file path CUC 

In [8]:
def find_graph_dir(base_dir: str = "/kaggle/input/lmtracker") -> str:
    direct = os.path.join(base_dir, "graph_data_torch.pt")
    if os.path.isfile(direct):
        return base_dir
    for root, dirs, files in os.walk(base_dir):
        if "graph_data_torch.pt" in files:
            return root
    raise FileNotFoundError("Không tìm thấy graph_data_torch.pt trong /kaggle/input/lmtracker")

dir_save = find_graph_dir()
print("📂 Using graph directory:", dir_save)

# HeteroData
data = torch.load(os.path.join(dir_save, "graph_data_torch.pt"), weights_only=False)
print(data)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("🖥️ Device:", device)

📂 Using graph directory: /kaggle/input/lmtracker/graph_data_20251119174904
HeteroData(
  (Computer, Connect, Computer)={ edge_index=[2, 215858] },
  (User, Logon, Computer)={ edge_index=[2, 266232] },
  (Computer, Logon_rev, User)={ edge_index=[2, 266232] },
  (User, SwitchUser, User)={ edge_index=[2, 5634] },
  (Computer, Create, Process)={ edge_index=[2, 1334355] },
  (Process, Create_rev, Computer)={ edge_index=[2, 1334355] }
)
🖥️ Device: cuda


## 2. Metadata & num_nodes cho từng node type


In [9]:
def infer_num_nodes_per_type(data):
    num_nodes = defaultdict(int)
    for (src, rel, dst), edge_index in data.edge_index_dict.items():
        row, col = edge_index
        if row.numel() > 0:
            num_nodes[src] = max(num_nodes[src], int(row.max().item()) + 1)
        if col.numel() > 0:
            num_nodes[dst] = max(num_nodes[dst], int(col.max().item()) + 1)
    return dict(num_nodes)

num_nodes_dict = infer_num_nodes_per_type(data)
print("num_nodes_dict:", num_nodes_dict)

metadata_raw = data.metadata()
print("metadata_raw:", metadata_raw)

# Nếu node_types rỗng, tự lấy từ num_nodes_dict
node_types = list(num_nodes_dict.keys())
edge_types = metadata_raw[1]
metadata = (node_types, edge_types)
print("✅ Fixed metadata:", metadata)


# Số node Computer (để sampling + build C–C graph)
num_nodes_computer = num_nodes_dict["Computer"]
print(f"\n📊 num_nodes_computer = {num_nodes_computer:,}")

# edge_index_dict trên GPU (cho model)
edge_index_dict_device = {
    k: v.to(device) for k, v in data.edge_index_dict.items()
}

num_nodes_dict: {'Computer': 15432, 'User': 11337, 'Process': 31026}
metadata_raw: ([], [('Computer', 'Connect', 'Computer'), ('User', 'Logon', 'Computer'), ('Computer', 'Logon_rev', 'User'), ('User', 'SwitchUser', 'User'), ('Computer', 'Create', 'Process'), ('Process', 'Create_rev', 'Computer')])
✅ Fixed metadata: (['Computer', 'User', 'Process'], [('Computer', 'Connect', 'Computer'), ('User', 'Logon', 'Computer'), ('Computer', 'Logon_rev', 'User'), ('User', 'SwitchUser', 'User'), ('Computer', 'Create', 'Process'), ('Process', 'Create_rev', 'Computer')])

📊 num_nodes_computer = 15,432


## 3. Sinh C–U–C path random (giống metapath CUC)


In [10]:
metapath = [
    ('Computer', 'Logon_rev', 'User'),
    ('User', 'Logon', 'Computer'),
]  # C -> U -> C

def generate_metapath_sampling(sample=20_000):
    """
    Tạo các path dạng C-U-C bằng random walk theo metapath trên CPU.
    Index trong path là local index theo từng node type (như file lmtracker).
    """
    # adj: (src,rel,dst) -> {src_id: [dst_ids]}
    adj = {}
    for (src, rel, dst), edge_index in data.edge_index_dict.items():
        row, col = edge_index
        for u, v in zip(row.tolist(), col.tolist()):
            adj.setdefault((src, rel, dst), {}).setdefault(int(u), []).append(int(v))

    start_type = metapath[0][0]  # 'Computer'
    start_nodes = list(range(num_nodes_computer))
    random.shuffle(start_nodes)

    paths = []
    print("\n🔄 Sampling C–U–C metapaths (CPU)...")
    for s in tqdm(start_nodes):
        if len(paths) >= sample:
            break
        cur = s
        seq = [int(cur)]
        ok = True
        for edge_key in metapath:
            neighbors = adj.get(edge_key, {}).get(int(cur), [])
            if not neighbors:
                ok = False
                break
            cur = random.choice(neighbors)
            seq.append(int(cur))
        if ok and len(seq) == 3:
            # seq = [C0, U1, C2]
            paths.append((seq[0], seq[1], seq[2]))

    paths = list(set(paths))
    print(f"✅ Generated {len(paths)} unique C-U-C paths (random)")
    return paths

random_paths = generate_metapath_sampling(sample=20_000)


🔄 Sampling C–U–C metapaths (CPU)...


100%|██████████| 15432/15432 [00:00<00:00, 360890.87it/s]

✅ Generated 15421 unique C-U-C paths (random)


## 4. Load các path CUC normal/malicious từ lmtracker

In [11]:

print("\n📥 Loading labeled C–U–C paths (CUC) from lmtracker...")

normal_path = list(torch.load(os.path.join(dir_save, "normal_path.pt"), weights_only=False))
malicious_all_path = list(torch.load(os.path.join(dir_save, "malicious_CUC_path.pt"), weights_only=False))
malicious_train_val_path = list(torch.load(os.path.join(dir_save, "malicious_train_val_path.pt"), weights_only=False))
malicious_test_path = list(torch.load(os.path.join(dir_save, "malicious_test_path.pt"), weights_only=False))

print(f"   normal_path: {len(normal_path)}")
print(f"   malicious_all_path: {len(malicious_all_path)}")
print(f"   malicious_train_val_path: {len(malicious_train_val_path)}")
print(f"   malicious_test_path: {len(malicious_test_path)}")

# Dataset dùng cho logistic regression: random_paths (normal) + malicious_all_path
all_paths_for_cls = random_paths + malicious_all_path
labels = [0] * len(random_paths) + [1] * len(malicious_all_path)
print(f"\n📊 Total paths for classification: {len(all_paths_for_cls)}")



📥 Loading labeled C–U–C paths (CUC) from lmtracker...
   normal_path: 483426
   malicious_all_path: 317
   malicious_train_val_path: 198
   malicious_test_path: 168

📊 Total paths for classification: 15738


## 5. Xây homogeneous graph C–C: edge (C0, C2) nếu tồn tại C–U–C

In [12]:
def build_computer_graph_from_paths(path_list_list):
    """
    path_list_list: list các list-path, mỗi path là (C, U, C)
    Trả về edge_index C–C (undirected) trên device.
    """
    edges_src = []
    edges_dst = []

    for paths in path_list_list:
        for (c0, u, c2) in paths:
            if c0 < 0 or c0 >= num_nodes_computer:
                continue
            if c2 < 0 or c2 >= num_nodes_computer:
                continue
            edges_src.append(c0)
            edges_dst.append(c2)
            # undirected
            edges_src.append(c2)
            edges_dst.append(c0)

    edge_index = torch.tensor([edges_src, edges_dst], dtype=torch.long, device=device)
    return edge_index

edge_index_cc = build_computer_graph_from_paths(
    [normal_path, random_paths, malicious_all_path]
)


print(f"\n🧱 C–C graph edges: {edge_index_cc.size(1):,}")


🧱 C–C graph edges: 998,328


## 6. Khởi tạo & train HeteroGraphSAGE trên đồ thị dị thể, loss trên C–C graph

In [13]:
model = HeteroGraphSAGE(
    metadata=metadata,
    num_nodes_dict=num_nodes_dict,
    hidden_channels=128,
    out_channels=128,
    num_layers=2,
    dropout=0.1,
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.003, weight_decay=1e-4)

total_params = sum(p.numel() for p in model.parameters())
print(f"✅ HeteroGraphSAGE params: {total_params:,}")


✅ HeteroGraphSAGE params: 7,842,048


In [14]:
def train_one_epoch_hetero(epoch: int, num_negative: int = 10, max_edges: int = 400_000):
    """
    - Forward trên Hetero graph (toàn bộ edge types).
    - Lấy embedding Computer.
    - Dùng contrastive loss trên C–C graph (giống code cũ).
    """
    model.train()
    optimizer.zero_grad()

    # 1) Hetero GraphSAGE: node embedding cho tất cả node types
    x_dict = model(edge_index_dict_device)   # dict[node_type] -> [N_nt, d]
    z_comp = x_dict["Computer"]              # [num_nodes_computer, d]

    # 2) Contrastive loss trên C–C graph
    src_all, dst_all = edge_index_cc
    num_edges = src_all.size(0)

    if max_edges is not None and num_edges > max_edges:
        perm = torch.randperm(num_edges, device=device)[:max_edges]
        src = src_all[perm]
        dst = dst_all[perm]
    else:
        src = src_all
        dst = dst_all

    # Positive pairs
    pos_src = z_comp[src]
    pos_dst = z_comp[dst]
    pos_scores = (pos_src * pos_dst).sum(dim=-1)
    pos_loss = -F.logsigmoid(pos_scores).mean()

    # Negative sampling
    neg_dst_idx = torch.randint(
        0,
        z_comp.size(0),
        (src.size(0) * num_negative,),
        device=device,
    )
    neg_src = pos_src.repeat_interleave(num_negative, dim=0)
    neg_dst = z_comp[neg_dst_idx]



    neg_scores = (neg_src * neg_dst).sum(dim=-1)
    neg_loss = -F.logsigmoid(-neg_scores).mean()

    loss = pos_loss + neg_loss
    loss.backward()
    optimizer.step()

    print(f"[hetero] Epoch {epoch:02d} | loss = {loss.item():.4f}")
    return loss.item()

print("\n🔥 Start training HeteroGraphSAGE (loss trên C–C graph)...")
start_time = time.time()
num_epochs = 80

for ep in range(1, num_epochs + 1):
    train_one_epoch_hetero(ep)

print(f"⏱️ Done training in {time.time() - start_time:.2f}s")


🔥 Start training HeteroGraphSAGE (loss trên C–C graph)...
[hetero] Epoch 01 | loss = 1.5946
[hetero] Epoch 02 | loss = 1.4815
[hetero] Epoch 03 | loss = 1.3937
[hetero] Epoch 04 | loss = 1.3638
[hetero] Epoch 05 | loss = 1.3526
[hetero] Epoch 06 | loss = 1.3251
[hetero] Epoch 07 | loss = 1.3107
[hetero] Epoch 08 | loss = 1.2959
[hetero] Epoch 09 | loss = 1.2842
[hetero] Epoch 10 | loss = 1.2645
[hetero] Epoch 11 | loss = 1.2460
[hetero] Epoch 12 | loss = 1.2326
[hetero] Epoch 13 | loss = 1.2226
[hetero] Epoch 14 | loss = 1.2163
[hetero] Epoch 15 | loss = 1.2107
[hetero] Epoch 16 | loss = 1.2068
[hetero] Epoch 17 | loss = 1.2019
[hetero] Epoch 18 | loss = 1.1989
[hetero] Epoch 19 | loss = 1.1953
[hetero] Epoch 20 | loss = 1.1936
[hetero] Epoch 21 | loss = 1.1920
[hetero] Epoch 22 | loss = 1.1899
[hetero] Epoch 23 | loss = 1.1877
[hetero] Epoch 24 | loss = 1.1871
[hetero] Epoch 25 | loss = 1.1858
[hetero] Epoch 26 | loss = 1.1840
[hetero] Epoch 27 | loss = 1.1835
[hetero] Epoch 28 | los

## 7. Lấy node embedding & sinh path embedding từ (C0, C2)

In [15]:
model.eval()
with torch.no_grad():
    x_dict = model(edge_index_dict_device)
    z_comp = x_dict["Computer"]  # [num_nodes_computer, d]

def batch_path_embeddings_from_CUC(paths, x_dict, desc: str, batch_size: int = 4096):
    """
    paths: list các path (C, U, C) với index local theo từng node type.
    x_dict: dict node_type -> embedding tensor từ model (Computer, User, ...)
    Trả về dict: path_tuple -> emb (mean(C0, U, C2), L2-normalized)
    """
    device = next(iter(x_dict.values())).device
    z_comp = x_dict["Computer"]
    z_user = x_dict["User"]

    res = {}
    n = len(paths)

    for i in tqdm(range(0, n, batch_size), desc=desc):
        batch = paths[i:i + batch_size]
        if not batch:
            continue

        # [B, 3] index cho (C0, U, C2)
        idx = torch.tensor(batch, dtype=torch.long, device=device)  # shape [B, 3]
        c0_idx = idx[:, 0]
        u_idx  = idx[:, 1]
        c2_idx = idx[:, 2]

        emb_c0 = z_comp[c0_idx]   # [B, d]
        emb_u  = z_user[u_idx]    # [B, d]
        emb_c2 = z_comp[c2_idx]   # [B, d]

        emb = (emb_c0 + emb_u + emb_c2) / 3.0
        emb = F.normalize(emb, p=2, dim=-1)

        for p, e in zip(batch, emb):
            res[tuple(p)] = e.detach().cpu()

    return res


print("\n🎯 Generating path embeddings from Computer embeddings...")

path_embedding = batch_path_embeddings_from_CUC(
    all_paths_for_cls,
    x_dict,
    desc="All paths for cls",
    batch_size=4096,
)


out_normal = batch_path_embeddings_from_CUC(
    normal_path,
    x_dict,
    desc="Normal paths",
    batch_size=4096,
)

out_mal_train_val = batch_path_embeddings_from_CUC(
    malicious_train_val_path,
    x_dict,
    desc="Malicious train/val",
    batch_size=4096,
)

out_mal_test = batch_path_embeddings_from_CUC(
    malicious_test_path,
    x_dict,
    desc="Malicious test",
    batch_size=4096,
)

print("\n✅ Example embedding shape:", next(iter(path_embedding.values())).shape)




🎯 Generating path embeddings from Computer embeddings...


Malicious test: 100%|██████████| 1/1 [00:00<00:00, 265.48it/s]


✅ Example embedding shape: torch.Size([128])


## 8. Lưu kết quả giống format MetaPath2Vec/GraphSAGE cũ 


In [16]:

datestring = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
cur_dir = os.getcwd()
store_directory = os.path.join(cur_dir, f"model_graphsage_CUC_hetero_{datestring}")
os.mkdir(store_directory)

# lưu state_dict thay vì full model object
torch.save(model.state_dict(), os.path.join(store_directory, "model_state.pt"))
torch.save(all_paths_for_cls, os.path.join(store_directory, "path.pt"))
torch.save(path_embedding, os.path.join(store_directory, "path_embedding.pt"))
torch.save(labels, os.path.join(store_directory, "path_labels.pt"))

torch.save(out_normal, os.path.join(store_directory, "out_normal.pt"))
torch.save(out_mal_train_val, os.path.join(store_directory, "out_mal_train_val.pt"))
torch.save(out_mal_test, os.path.join(store_directory, "out_mal_test.pt"))

print(f"\n💾 Saved all outputs to: {store_directory}")


💾 Saved all outputs to: /kaggle/working/model_graphsage_CUC_hetero_20251130081411
